In [1]:
from pathlib import Path
import torch
from torch_geometric.data import Data

In [2]:
def load_train_graphs(root: Path):
    graphs = []
    for pt_file in sorted(root.glob("*.pt")):
        data = torch.load(pt_file, weights_only=False)
        graphs.append(data)
        print(pt_file)
    return graphs

In [3]:
root = Path("Datasets/train_pyg")
graphs = load_train_graphs(root)

print(f"Total graphs loaded: {len(graphs)}")
print(graphs[0])

Datasets\train_pyg\00000.pt
Datasets\train_pyg\00001.pt
Datasets\train_pyg\00002.pt
Datasets\train_pyg\00003.pt
Datasets\train_pyg\00004.pt
Datasets\train_pyg\00005.pt
Datasets\train_pyg\00006.pt
Datasets\train_pyg\00007.pt
Datasets\train_pyg\00008.pt
Datasets\train_pyg\00009.pt
Datasets\train_pyg\00010.pt
Datasets\train_pyg\00011.pt
Datasets\train_pyg\00012.pt
Datasets\train_pyg\00013.pt
Datasets\train_pyg\00014.pt
Datasets\train_pyg\00015.pt
Datasets\train_pyg\00016.pt
Datasets\train_pyg\00017.pt
Datasets\train_pyg\00018.pt
Datasets\train_pyg\00019.pt
Datasets\train_pyg\00020.pt
Datasets\train_pyg\00021.pt
Datasets\train_pyg\00022.pt
Datasets\train_pyg\00023.pt
Datasets\train_pyg\00024.pt
Datasets\train_pyg\00025.pt
Datasets\train_pyg\00026.pt
Datasets\train_pyg\00027.pt
Datasets\train_pyg\00028.pt
Datasets\train_pyg\00029.pt
Datasets\train_pyg\00030.pt
Datasets\train_pyg\00031.pt
Datasets\train_pyg\00032.pt
Datasets\train_pyg\00033.pt
Datasets\train_pyg\00034.pt
Datasets\train_pyg\0

In [4]:
from torch.utils.data import Dataset

class TSPTrainDataset(Dataset):
    def __init__(self, graphs):
        self.graphs = graphs

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        return self.graphs[idx]

dataset = TSPTrainDataset(graphs)
print(f"Dataset length: {len(dataset)}")

Dataset length: 4165


In [5]:
from torch_geometric.loader import DataLoader

batch_size = 32  # pots ajustar-ho més endavant
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# comprovació ràpida
batch = next(iter(loader))
print(batch)

DataBatch(x=[4211, 2], edge_index=[2, 936112], edge_attr=[936112, 1], y=[32], node_id=[4211], batch=[4211], ptr=[33])


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GraphConv

class GNN(nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        self.conv1 = GraphConv(in_channels, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, hidden_channels)
        self.lin = nn.Linear(hidden_channels, 1)  # score per node

    def forward(self, x, edge_index, edge_attr, batch):
        # edge_attr: [num_edges, 1] → edge_weight: [num_edges]
        edge_weight = edge_attr.squeeze(-1)

        # Message passing amb edge_weight
        x = self.conv1(x, edge_index, edge_weight=edge_weight)
        x = F.relu(x)

        x = self.conv2(x, edge_index, edge_weight=edge_weight)
        x = F.relu(x)

        # Sortida per node
        out = self.lin(x).squeeze(-1)  # [num_nodes]

        return out

In [ ]:
import torch

in_channels = graphs[0].x.shape[1]      # 2
hidden_channels = 64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = GNN(in_channels, hidden_channels).to(device)
print(model)

GNN(
  (conv1): GraphConv(2, 64)
  (conv2): GraphConv(64, 64)
  (lin): Linear(in_features=64, out_features=1, bias=True)
)


In [9]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0

    for batch in loader:
        batch = batch.to(device)

        # Forward
        out = model(batch.x, batch.edge_index, batch.edge_attr, batch.batch)

        # out: [num_nodes]
        # batch.y: scalar → fem unsqueeze per fer-lo compatible
        loss = criterion(out.unsqueeze(0), batch.y.unsqueeze(0))

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

loss = train_one_epoch(model, loader, optimizer, criterion, device)
print("Loss:", loss)

RuntimeError: 0D or 1D target tensor expected, multi-target not supported